Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

Choose from alternatives

In [ ]:
# Yeast 8
model = cobra.io.read_sbml_model("./models/yeast-GEM-BiGG.xml")

In [ ]:
# Enzyme-constrained Yeast 8, batch
# https://github.com/SysBioChalmers/ecModels/tree/main/ecYeastGEM/model
model = cobra.io.read_sbml_model("./models/ecYeastGEM_batch.xml")

Show model

In [ ]:
model

# Utilities

In [ ]:
def print_formulas(reaction):
    """Print formulas of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): F {reactant.formula}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): F {product.formula}')

def print_formula_weights(reaction):
    """Print formula weights of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): MW {reactant.formula_weight}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): MW {product.formula_weight}')

# Objective function

In the ecYeast8 (batch) model, the objective function -- growth -- is reaction ID `r_2111`.

This reaction is linked to the biomass reaction, ID `r_4041`.

Here, we also see the stoichiometry.  There are five classes of macromolecules: lipids, proteins, carbohydrates, DNA, and RNA.  And there are two other bulk metabolites: cofactor and ion.

In [ ]:
model.reactions.get_by_id('r_2111')

In [ ]:
model.reactions.get_by_id('r_4041')

Medium

In [ ]:
model.medium

In [ ]:
for reaction_id in model.medium.keys():
    print(model.reactions.get_by_id(reaction_id).name)

Remove bounds on glucose uptake and growth rate

In [ ]:
# (no need because bounds are already unrestricted)
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000.0, 0)
# Unrestrict oxygen uptake (aerobic)
model.reactions.get_by_id('r_1992').bounds = (-1000.0, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_4041').bounds = (0, 1000.0)

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

# Computing molecular weights of bulk metabolites

The ecYeast8 model does not specify the molecular weights of these bulk metabolites.

To compute these, I assumed conservation of mass, i.e.

\begin{equation}
    \sum_{s}(\text{molar mass}_{s})(\text{stoichiometric coefficient}_{s}) - \sum_{p}(\text{molar mass}_{p})(\text{stoichiometric coefficient}_{p})
\end{equation}

where $s = 1, ... (\text{number of substrates})$ represents substrates and $p = 1, ... (\text{number of products})$ represents products of the reaction in question.

This procedure must be applied because the ecYeast8 model does not necessarily imply that
each molecule of macromolecule corresponds to a single molecule in a real cell.

## Carbohydrates

To compute the molecular weight of the carbohydrate metabolite, I inspected reaction r_4048.  This reaction accounts for structural (e.g. cell wall) and storage carbohydrates.

In [ ]:
model.reactions.get_by_id('r_4048')

Here, the molecular weights of all species except for carbohydrate, the bulk metabolite, are
represented in the model.

In [ ]:
print_formula_weights(model.reactions.get_by_id('r_4048'))

Thus, the conservation of mass can be applied directly.

In [ ]:
MW_CARB = -sum(
    [metabolite.formula_weight * coeff
     for metabolite, coeff in model.reactions.get_by_id('r_4048').metabolites.items()]
)
print(MW_CARB)

We will re-use this code a lot, so I will write a convenience function:

In [ ]:
# I admit that this function is not generalisable, but this notebook is
# a quick-and-dirty idea sandbox... for now.
def mw_from_reaction(reaction):
    """
    Computes molecular weight of a species of unknown weight.
    
    Only works if there is just one species with unknown weight.
    Assumes that the stoichiometric coefficient of the species is 1.
    """
    return -sum(
        [metabolite.formula_weight * coeff
         for metabolite, coeff in reaction.metabolites.items()]
    )

## DNA, RNA, cofactor, ion

The same process can be applied to compute the molecular weights of the DNA, RNA,
cofactor, and ion metabolites.  This is because the equations are similar.  They have reactants with molecular weights represented in the model.  And only the bulk metabolite, the sole product, as the metabolite with an unspecified molecular weight. 

In [ ]:
# DNA
print_formula_weights(model.reactions.get_by_id('r_4050'))

In [ ]:
# RNA
print_formula_weights(model.reactions.get_by_id('r_4049'))

In [ ]:
# cofactor
print_formula_weights(model.reactions.get_by_id('r_4598'))

In [ ]:
# ion
print_formula_weights(model.reactions.get_by_id('r_4599'))

In [ ]:
# The bulk metabolite has a stoichiometric coefficient of 1,
# so mw_from_reaction can be used directly.
MW_DNA = mw_from_reaction(model.reactions.get_by_id('r_4050'))
MW_RNA = mw_from_reaction(model.reactions.get_by_id('r_4049'))
MW_COFACTOR = mw_from_reaction(model.reactions.get_by_id('r_4598'))
MW_ION = mw_from_reaction(model.reactions.get_by_id('r_4599'))

print(MW_DNA)
print(MW_RNA)
print(MW_COFACTOR)
print(MW_ION)

Note: I don't know why DNA and RNA molecular weights differ by an order of magnitude.  The stochiometric constants for the RNA pseudoreaction are an order of magnitude greater than that of DNA, and I don't know why.

**Answer: They reflect relative abundances**

## Protein

This is slightly less straightforward because the aminoacyl-tRNA reactants are represented in the form of the atoms that make up the aminoacyl residues plus R to represent the tRNA, and the tRNA products are represented as RH.

In [ ]:
print_formulas(model.reactions.get_by_id('r_4047'))

The problem is: R is not listed as an element in `cobrapy`, so I can't use built-in functions (i.e. `print_formula_weights` breaks).  Therefore, I reverse-engineered `cobra.core.formula` and `cobra.core.metabolite` so it can deal with an 'R' element.

In [ ]:
# hack: reverse-engineering cobra.core.formula and cobra.core.metabolite
# so it can deal with an 'R' element
import re
from typing import TYPE_CHECKING, Dict, Optional, Union
from cobra.core.formula import elements_and_molecular_weights

element_re = re.compile("([A-Z][a-z]?)([0-9.]+[0-9.]?|(?=[A-Z])?)")
elements_and_molecular_weights['R'] = 0

def elements(formula) -> Optional[Dict[str, Union[int, float]]]:
    """Get dicitonary of elements and counts.

    Dictionary of elements as keys and their count in the metabolite
    as integer. When set, the `formula` property is updated accordingly.

    Returns
    -------
    composition: None or Dict
        A dictionary of elements and counts, where count is int unless it is needed
        to be a float.
        Returns None in case of error.

    """
    tmp_formula = formula
    if tmp_formula is None:
        return {}
    # necessary for some old pickles which use the deprecated
    # Formula class
    tmp_formula = str(formula)
    # commonly occurring characters in incorrectly constructed formulas
    if "*" in tmp_formula:
        warn(f"invalid character '*' found in formula '{formula}'")
        tmp_formula = tmp_formula.replace("*", "")
    if "(" in tmp_formula or ")" in tmp_formula:
        warn(f"invalid formula (has parenthesis) in '{formula}'")
        return None
    composition = {}
    parsed = element_re.findall(tmp_formula)
    for element, count in parsed:
        if count == "":
            count = 1
        else:
            try:
                count = float(count)
                int_count = int(count)
                if count == int_count:
                    count = int_count
                else:
                    warn(f"{count} is not an integer (in formula {formula})")
            except ValueError:
                warn(f"failed to parse {count} (in formula {formula})")
                return None
        if element in composition:
            composition[element] += count
        else:
            composition[element] = count
    return composition

def formula_weight(elements) -> Union[int, float]:
    """Calculate the formula weight.

    Returns
    ------
    float, int
        Weight of formula, based on the weight and count of elements. Can be int if
        the formula weight is a whole number, but unlikely.
    """
    try:
        return sum(
            [
                count * elements_and_molecular_weights[element]
                for element, count in elements.items()
            ]
        )
    except KeyError as e:
        warn(f"The element {e} does not appear in the periodic table")

Fortunately, the protein bulk metabolite is the only product with an unknown molecular weight, so I can use the same approach as before to compute the molecular weight.

In [ ]:
protein_pseudoreaction = model.reactions.get_by_id('r_4047')
    
MW_PROTEIN = -sum(
    [formula_weight(elements(metabolite.formula)) * coeff
     for metabolite, coeff in protein_pseudoreaction.metabolites.items()]
)
print(MW_PROTEIN)

## Lipids

Finally, the lipid metabolite is the least straightforward because some of the reactants do not
have molecular weights specified. The lipid pseudoreaction is represented in reaction r_2108:

In [ ]:
model.reactions.get_by_id('r_2108')

Both `lipid backbone` and `lipid chain` have no molecular weight specified.

In [ ]:
print_formula_weights(model.reactions.get_by_id('r_2108'))

### Lipid chain

Reaction r_4065 specifies a lipid chain pseudoreaction, in which lipid chain is generated:

In [ ]:
print_formula_weights(model.reactions.get_by_id('r_4065'))

As all reactants have molecular weights defined in the model, the molecular weight of lipid
chain can be computed from the mass balance of this reaction.

In [ ]:
MW_LIPID_CHAIN = mw_from_reaction(model.reactions.get_by_id('r_4065'))
print(MW_LIPID_CHAIN)

### Lipid backbone

Reaction r_4063 specifies a lipid backbone pseudoreaction, in which lipid backbone is
generated:

In [ ]:
print_formula_weights(model.reactions.get_by_id('r_4063'))

Within this reaction, all reactants have defined molecular weights except for `fatty acid
backbone`. Four reactions in the model produce `fatty acid backbone`.

In [ ]:
fab_reaction_list = ['r_3975', 'r_3976', 'r_3977', 'r_3978']
mw_fab_list = []

for fab_reaction_id in fab_reaction_list:
    print(f'ID: {fab_reaction_id}')
    fab_reaction = model.reactions.get_by_id(fab_reaction_id)
    print(f'Reaction: {fab_reaction.reaction}')
    mw = mw_from_reaction(fab_reaction)
    # Stoichiometric coefficient of fatty acid backbone is not 1
    # in these reactions
    mw /= fab_reaction.metabolites[model.metabolites.get_by_id('s_0694[c]')]
    print(f'Computed molecular weight: {mw}')
    mw_fab_list.append(mw)
    print('\n')

Note: the molecular weights computed from each equation
are different, as shown above. Since the differences are slight, and ultimately I
am making a back-of-the-envelope calculation, I took the average of the four weights.

In [ ]:
MW_FATTY_ACID_BACKBONE = np.mean(mw_fab_list)
print(MW_FATTY_ACID_BACKBONE)

Now, I feed this number back into the lipid backbone pseudoreaction.

In [ ]:
# I can do this because the stoichiometric constant of
# the lipid backbone bulk metabolite is 1.
MW_LIPID_BACKBONE = 0
for metabolite, coeff in model.reactions.get_by_id('r_4063').metabolites.items():
    if metabolite.id == 's_0694':
        MW_LIPID_BACKBONE += coeff * MW_FATTY_ACID_BACKBONE
    else:
        MW_LIPID_BACKBONE += coeff * metabolite.formula_weight
MW_LIPID_BACKBONE = -MW_LIPID_BACKBONE
print(MW_LIPID_BACKBONE)

### Altogether

In [ ]:
MW_LIPID = MW_LIPID_BACKBONE + MW_LIPID_CHAIN
print(MW_LIPID)

## Biomass

The molecular weight of biomass is simply the molecular weights of each bulk metabolite added together.

Note that H2O, ATP, ADP, and Pi are involved in the reaction too.  But, as they are already mass-balanced, they can be ignored in this calculation.

In [ ]:
MW_BIOMASS = MW_PROTEIN + MW_CARB + MW_RNA + MW_LIPID + MW_COFACTOR + MW_DNA + MW_ION
print(MW_BIOMASS)

# Modify biomass reaction by ablating each type of macromolecule

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeastGEM_batch.xml")

## Some convenience classes

In [ ]:
CELL_DRY_MASS = 15e-12 # g

class BiomassComponent():
    def __init__(
        self,
        metabolite_label,
        metabolite_id,
        pseudoreaction,
        molecular_mass,
        mass_per_cell,
        copy_number,
    ):
        self.metabolite_label = metabolite_label
        self.metabolite_id = metabolite_id
        self.pseudoreaction = pseudoreaction
        self.molecular_mass = molecular_mass # g/mmol
        self.mass_per_cell = mass_per_cell # g
        self.copy_number = copy_number
        
        self.ablated_flux = None # h-1
        self.est_time = None # h
        
    def get_est_time(self):
        self.est_time = 1/self.ablated_flux        

We use molecular weights calculated earlier and other attributes from the cell economics project.

In [ ]:
MW_CARB = 368.03795704972003
MW_DNA = 3.9060196439999997
MW_RNA = 64.04235752722991
MW_PROTEIN = 504.3744234012359
MW_COFACTOR = 4.832782477018401
MW_ION = 2.4815607543700002
MW_LIPID = 31.5659867112958
MW_BIOMASS = 979.24108756487

In [ ]:
# TODO:
# - Create CSV table containing these
# - Create a class builder that builds these classes based on the CSV table
# - FURTHER: make it able to deal with ranges of values (lower limit, upper limit)

Lipids = BiomassComponent(
    metabolite_label='lipid',
    metabolite_id='s_1096[c]',
    pseudoreaction='r_2108',
    molecular_mass=MW_LIPID*1e-3,
    mass_per_cell=900e-15,
    copy_number=1e9,
)

Proteins = BiomassComponent(
    metabolite_label='protein',
    metabolite_id='s_3717[c]',
    pseudoreaction='r_4047',
    molecular_mass=MW_PROTEIN*1e-3,
    mass_per_cell=7650e-15,
    copy_number=1e8,
)

Carbohydrates = BiomassComponent(
    metabolite_label='carbohydrate',
    metabolite_id='s_3718[c]',
    pseudoreaction='r_4048',
    molecular_mass=MW_CARB*1e-3,
    mass_per_cell=(75+3450)*1e-15, # 'storage carbohydrates' + 'structural polymers'
    copy_number=2122804981, # estimated from above & avogadro's const
)

DNA = BiomassComponent(
    metabolite_label='DNA',
    metabolite_id='s_3720[c]',
    pseudoreaction='r_4050',
    molecular_mass=MW_DNA*1e-3,
    mass_per_cell=75e-15,
    copy_number=16,
)

RNA = BiomassComponent(
    metabolite_label='RNA',
    metabolite_id='s_3719[c]',
    pseudoreaction='r_4049',
    molecular_mass=MW_RNA*1e-3,
    mass_per_cell=1650e-15,
    copy_number=4e6,
)

Cofactors = BiomassComponent(
    metabolite_label='cofactor',
    metabolite_id='s_4205[c]',
    pseudoreaction='r_4598',
    molecular_mass=MW_COFACTOR*1e-3,
    mass_per_cell=1,
    copy_number=1,
)

Ions = BiomassComponent(
    metabolite_label='ion',
    metabolite_id='s_4206[c]',
    pseudoreaction='r_4599',
    molecular_mass=MW_ION*1e-3,
    mass_per_cell=1,
    copy_number=1,
)

Copy model

In [ ]:
model_saved = model.copy()

Simulate

In [ ]:
from cobra.util.solver import linear_reaction_coefficients

# Set up lists
biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA, Cofactors, Ions]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

all_pseudoreaction_ids = [
    (biomass_component.metabolite_label, biomass_component.pseudoreaction)
    for biomass_component in biomass_component_list
]
all_pseudoreaction_ids.append(('biomass', 'r_4041'))
all_pseudoreaction_ids.append(('objective', 'r_2111'))
    
def barplot_fluxes(pfba_solution, all_pseudoreactions_ids, plot_title):
    plt.subplots()
    plt.bar(
        [label for (label, _) in all_pseudoreaction_ids],
        [pfba_solution.fluxes[pseudoreaction_id]
         for (_, pseudoreaction_id) in all_pseudoreaction_ids],
    )
    plt.ylim((0,3))
    plt.title(plot_title)
    plt.xticks(rotation=45, ha="right")
    
# ORIGINAL

model = model_saved.copy()
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_2111').bounds = (0, 1000)
# Optimise using parsimonious FBA
pfba_solution = cobra.flux_analysis.pfba(model)

# Outputs
barplot_fluxes(pfba_solution, all_pseudoreaction_ids, plot_title='original')
print(f'Flux: {pfba_solution.fluxes["r_2111"]} h-1')
biomass_time = 1/(pfba_solution.fluxes['r_2111'])
print(f'Estimated time: {biomass_time:.4f} hours')
print('\n')

# ABLATED

for biomass_component in biomass_component_list:
    print(f'Prioritising {biomass_component.metabolite_label}')
    model = model_saved.copy()
    model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
    model.reactions.get_by_id('r_2111').bounds = (0, 1000)
    
    # boilerplate: lookup
    to_ablate = all_metabolite_ids.copy()
    to_ablate.remove(biomass_component.metabolite_id)
    to_ablate_keys = [
        model.metabolites.get_by_id(metabolite_id)
        for metabolite_id in to_ablate
    ]
    to_ablate_dict = dict(zip(to_ablate_keys, [-1]*len(to_ablate_keys)))
    
    # ablate metabolites from biomass reaction
    model.reactions.get_by_id('r_4041').subtract_metabolites(to_ablate_dict)
    
    # change objective
#    model.objective = biomass_component.pseudoreaction
#    print(linear_reaction_coefficients(model))
    
    # optimise model
    #pfba_solution = cobra.flux_analysis.pfba(model)
    fba_solution = model.optimize()
    
    # Outputs
    biomass_component.ablated_flux = fba_solution.fluxes["r_2111"]

    barplot_fluxes(
        fba_solution,
        all_pseudoreaction_ids,
        plot_title=f'Prioritising {biomass_component.metabolite_label}'
    )
    print(f'Flux: {fba_solution.fluxes["r_2111"]} h-1')
    biomass_component.get_est_time()
    print(f'Estimated time: {biomass_component.est_time:.4f} hours')
    print('\n')
    
total_time = sum([biomass_component.est_time for biomass_component in biomass_component_list])
print(f'sum of times: {total_time:.4f} hours')

# Demand reactions

In [ ]:
model = model_saved

biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA, Cofactors, Ions]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

# Create demand reactions for all bulk pseudometabolites
for metabolite_id in all_metabolite_ids:
    model.add_boundary(model.metabolites.get_by_id(metabolite_id), type='demand')
    
# Set exchange reaction bounds
model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
model.reactions.get_by_id('r_2111').bounds = (0, 1000)

In [ ]:
model.reactions.get_by_id('DM_s_1096[c]')

In [ ]:
for biomass_component in biomass_component_list:
    demand_reaction_id = 'DM_' + biomass_component.metabolite_id
    print(f'{demand_reaction_id} ({biomass_component.metabolite_label})')
    model.objective = demand_reaction_id
    pfba_solution = cobra.flux_analysis.pfba(model)
    print(f'Demand rxn flux: {pfba_solution.fluxes[demand_reaction_id]} h-1')
    print(f'Growth rate: {pfba_solution.fluxes["r_4041"]} h-1')
    time = 1/(pfba_solution.fluxes[demand_reaction_id])
    print(f'Time: {time} h')